In [ ]:

import csv
from datetime import datetime, timedelta, date
from calendar import monthrange

# Global variables
DEFAULT_MESS = {
    "Shantanu Deelip Dewale": "North Mess",
    "Dasari Sheikinah Myrtle": "South Mess",
    "Hrishikesh Dilip Ghorpade": "Yukthahar",
    "Yadava Nandini": "Yukthahar",
    "Vantepaka Akshay Rathan": "South Mess",
    "Chedapangu Seema": "Kadamba Veg Mess",
    "Jayant Lodhi": "North Mess",
    "Medari Vikas": "North Mess",
    "Katta Kavya": "Kadamba Veg Mess",
    "Abhishek Tiwari": "North Mess",
    "Baddipadiga Shruthi": "South Mess",
    "Meera P": "Yukthahar"
}

registrations = []  # List to store registrations of mess
cancellations = []  # List to store cancellations of mess

# Load users data from CSV file
def load_users_from_csv(file_name="user_data.csv"):
    users_dict = {}
    try:
        with open(file_name, mode="r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                user_id = row["User ID"]
                users_dict[user_id] = {
                    "password": row["Password"],
                    "roll_number": row["Roll Number"],
                    "role": row["Role"]
                }
    except FileNotFoundError:
        print(f"Error: {file_name} not found.")
    return users_dict

# Load mess menu from CSV file
def load_menu_from_csv(file_name="mess_menu.csv"):
    menu_dict = {}
    try:
        with open(file_name, mode="r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                mess_name = row["Mess Name"]
                day = row["Day"]
                meal_type = row["Meal Type"]
                menu_items = row["Menu Items"]

                if mess_name not in menu_dict:
                    menu_dict[mess_name] = {}
                if day not in menu_dict[mess_name]:
                    menu_dict[mess_name][day] = {}
                
                menu_dict[mess_name][day][meal_type] = menu_items
    except FileNotFoundError:
        print(f"Error: {file_name} not found.")
    return menu_dict

# Load mess rates from CSV file
def load_mess_rates_from_csv(file_name="mess_rates.csv"):
    mess_rates = {}
    try:
        with open(file_name, mode="r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                mess_name = row["Mess Name"]
                try:
                    rate = float(row["Rate"])
                    mess_rates[mess_name] = rate
                except ValueError:
                    print(f"Invalid rate for {mess_name} in {file_name}. Setting rate to 0.")
                    mess_rates[mess_name] = 0
    except FileNotFoundError:
        print(f"Error: {file_name} not found. Please ensure it exists and is formatted correctly.")
    return mess_rates

# Load registrations of mess from CSV file
def load_registrations_from_csv(file_name="registrations.csv"):
    global registrations
    try:
        with open(file_name, mode="r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                try:
                    registration = {
                        "user_id": row["User ID"],
                        "mess": row["Mess"],
                        "registration_type": row["Registration Type"],
                        "start_date": datetime.strptime(row["Start Date"], "%Y-%m-%d").date(),
                        "end_date": datetime.strptime(row["End Date"], "%Y-%m-%d").date()
                    }
                    registrations.append(registration)
                except ValueError as ve:
                    print(f"Error parsing registration row: {row}. Error: {ve}")
    except FileNotFoundError:
        print(f"No existing registrations found in {file_name}. Starting fresh.")

# Load cancellations of mess from CSV file
def load_cancellations_from_csv(file_name="cancellations.csv"):
    global cancellations
    try:
        with open(file_name, mode="r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                try:
                    cancellation = {
                        "user_id": row["User ID"],
                        "mess": row["Mess"],
                        "start_date": datetime.strptime(row["Start Date"], "%Y-%m-%d").date(),
                        "end_date": datetime.strptime(row["End Date"], "%Y-%m-%d").date()
                    }
                    cancellations.append(cancellation)
                except ValueError as ve:
                    print(f"Error parsing cancellation row: {row}. Error: {ve}")
    except FileNotFoundError:
        print(f"No existing cancellations found in {file_name}. Starting fresh.")

# Save new registrations of mess to CSV file
def save_registrations_to_csv(file_name="registrations.csv"):
    try:
        with open(file_name, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["User ID", "Mess", "Registration Type", "Start Date", "End Date"])
            for reg in registrations:
                writer.writerow([
                    reg["user_id"],
                    reg["mess"],
                    reg["registration_type"],
                    reg["start_date"].strftime("%Y-%m-%d"),
                    reg["end_date"].strftime("%Y-%m-%d")
                ])
        print(f"Registrations saved to {file_name} successfully.")
    except IOError:
        print(f"Error: Unable to save registrations to {file_name}.")

# Save cancellations of mess to CSV file
def save_cancellations_to_csv(file_name="cancellations.csv"):
    try:
        with open(file_name, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["User ID", "Mess", "Start Date", "End Date"])
            for cancel in cancellations:
                writer.writerow([
                    cancel["user_id"],
                    cancel["mess"],
                    cancel["start_date"].strftime("%Y-%m-%d"),
                    cancel["end_date"].strftime("%Y-%m-%d")
                ])
        print(f"Cancellations saved to {file_name} successfully.")
    except IOError:
        print(f"Error: Unable to save cancellations to {file_name}.")

# Display the system time
def timed_input(prompt):
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[System Time: {current_time}]")
    return input(prompt)

# Set login credentials
def login(users):
    print("---- Login ----")
    user_id = input("Enter User ID: ")
    password = input("Enter Password: ")
    if user_id in users and users[user_id]["password"] == password:
        print(f"Welcome, {user_id}!")
        return users[user_id]["role"], user_id
    else:
        print("Invalid credentials. Please try again.")
        return None, None

# View mess menu
def view_mess_menu(mess_data):
    print("\nAvailable Mess Menus:")
    for mess, days in mess_data.items():
        print(f"\nMess: {mess}")
        for day, meals in days.items():
            print(f"{day}: {meals}")

def register_for_mess(user_id, mess_data):
    global cancellations,registrations 
    print("\n--- Register for a Mess ---")
    print("Available Messes:")
    for mess in mess_data.keys():
        print(f"- {mess}")
    
    mess = input("Enter the mess name you want to register for: ")
    if mess not in mess_data:
        print("Invalid mess name. Registration failed.")
        return

    current_date = date.today()

    # Daily Registration
    try:
        start_date = datetime.strptime(input("Enter the start date (YYYY-MM-DD): "), "%Y-%m-%d").date()
        end_date = datetime.strptime(input("Enter the end date (YYYY-MM-DD): "), "%Y-%m-%d").date()

        # Enforce the 2-day rule for mess registration
        if (start_date - current_date).days < 2:
            print("Changing mess registration is permitted two days in advance only.")
            return

        if start_date < current_date or end_date < start_date:
            print("Invalid date range. Ensure the start date is today or later, and the end date is after the start date.")
            return
        
        # Add daily registration of mess
        new_registration = {
            "user_id": user_id,
            "mess": mess,
            "registration_type": "daily",
            "start_date": start_date,
            "end_date": end_date,
        }
        registrations.append(new_registration)
        print(f"Successfully registered for {mess} from {start_date.strftime('%d %B %Y')} to {end_date.strftime('%d %B %Y')}.")

        # Remove overlapping cancellations of mess
        updated_cancellations = []
        for cancel in cancellations:
            if cancel["user_id"] == user_id:
                # Check if cancellation overlaps with newly registered mess
                cancel_start = cancel["start_date"]
                cancel_end = cancel["end_date"]
                reg_start = new_registration["start_date"]
                reg_end = new_registration["end_date"]

                if cancel_end < reg_start or cancel_start > reg_end:
                    # No overlap, keep the cancelled mess
                    updated_cancellations.append(cancel)
                else:
                    # Overlaps, adjust or remove the cancellation of mess
                    if cancel_start < reg_start:
                        if cancel_end > reg_end:
                            updated_cancellations.append({
                                "user_id": user_id,
                                "mess": cancel["mess"],
                                "start_date": cancel_start,
                                "end_date": reg_start - timedelta(days=1)
                            })
                            updated_cancellations.append({
                                "user_id": user_id,
                                "mess": cancel["mess"],
                                "start_date": reg_end + timedelta(days=1),
                                "end_date": cancel_end
                            })
                        else:
                            updated_cancellations.append({
                                "user_id": user_id,
                                "mess": cancel["mess"],
                                "start_date": cancel_start,
                                "end_date": reg_start - timedelta(days=1)
                            })
                    elif cancel_end > reg_end:
                        updated_cancellations.append({
                            "user_id": user_id,
                            "mess": cancel["mess"],
                            "start_date": reg_end + timedelta(days=1),
                            "end_date": cancel_end
                        })
            else:
                updated_cancellations.append(cancel)
        cancellations = updated_cancellations

    except ValueError:
        print("Invalid registration.")

# Function to cancel registration of mess
def cancel_registration(user_id):
    global registrations, cancellations
    try:
        current_date = date.today()
        min_cancellation_date = current_date + timedelta(days=1)

        year = int(input("Enter the year (YYYY): "))
        month = int(input("Enter the month (MM): "))
        start_day = int(input("Enter the start day (DD): "))
        end_day = int(input("Enter the end day (DD): "))
        
        # Set the start and end dates of registrations and cancellations
        start_date = date(year, month, start_day)
        end_date = date(year, month, end_day)
        
        # Validating that end date is not before the start date
        if start_date > end_date:
            print("Error: Start date cannot be after end date.")
            return

        # Check that the start_date is at least 1 days from current date
        if start_date < min_cancellation_date:
            print("Error: Cancellations is permitted at least 1 days in advance.")
            return

        # Check for past dates
        if start_date < current_date:
            print("Error: Cancellation not allowed.")
            return

        # Process cancellations
        updated_registrations = []
        cancellation_made = False

        for reg in registrations:
            if reg["user_id"] == user_id:
                reg_start_date = reg.get("start_date")
                reg_end_date = reg.get("end_date")
                reg_type = reg.get("registration_type", "unknown")
                
                if reg_start_date and reg_end_date and reg_start_date <= end_date and reg_end_date >= start_date:
                    # Overlapping range; remove specific days from the registration
                    if reg_start_date < start_date:
                        updated_registrations.append({
                            "user_id": reg["user_id"],
                            "mess": reg["mess"],
                            "registration_type": reg["registration_type"],
                            "start_date": reg_start_date,
                            "end_date": start_date - timedelta(days=1)
                        })
                    
                    if reg_end_date > end_date:
                        updated_registrations.append({
                            "user_id": reg["user_id"],
                            "mess": reg["mess"],
                            "registration_type": reg["registration_type"],
                            "start_date": end_date + timedelta(days=1),
                            "end_date": reg_end_date
                        })
                    
                    # Record the cancellation
                    cancellations.append({
                        "user_id": user_id,
                        "mess": reg["mess"],
                        "start_date": start_date,
                        "end_date": end_date
                    })
                    cancellation_made = True
                else:
                    # No overlap, keep this registration
                    updated_registrations.append(reg)
            else:
                # Not the same user, keep this registration
                updated_registrations.append(reg)
        
        registrations = updated_registrations

        if cancellation_made:
            print(f"Mess registration canceled for {user_id} from {start_date.strftime('%d-%m-%Y')} to {end_date.strftime('%d-%m-%Y')}.")
        else:
            print("No matching registrations found to cancel for the mentioned period.")
    except ValueError:
        print("Invalid input. Please ensure all dates are entered correctly.")

# View registered mess for the month with rates
def view_mess_for_month(user_id, default_mess, mess_rates):
    try:
        year = int(input("Enter the year (YYYY): "))
        month = int(input("Enter the month (MM): "))
        # Validate month
        if not (1 <= month <= 12):
            print("Invalid month entered. Please enter a valid month (1-12).")
            return
        
        # Get the last day of the month
        _, last_day = monthrange(year, month)
    except ValueError:
        print("Invalid year or month input.")
        return
    
    # Initializing the mess schedule
    mess_schedule = {day: default_mess for day in range(1, last_day + 1)}

    # Start mess registrations
    for reg in registrations:
        if reg["user_id"] == user_id:
            start_date = reg.get("start_date")
            end_date = reg.get("end_date")
            mess_name = reg.get("mess", default_mess)
            
            # Check for overlaps with the current month
            if start_date <= datetime(year, month, last_day).date() and end_date >= datetime(year, month, 1).date():
                for day in range(
                    max(start_date.day, 1) if start_date.month == month else 1,
                    min(end_date.day, last_day) + 1 if end_date.month == month else last_day + 1
                ):
                    mess_schedule[day] = mess_name

    # Start mess cancellations
    for cancel in cancellations:
        if cancel["user_id"] == user_id:
            cancel_start = cancel.get("start_date")
            cancel_end = cancel.get("end_date")
            
            # Check for overlaps with the current month
            if cancel_start <= datetime(year, month, last_day).date() and cancel_end >= datetime(year, month, 1).date():
                for day in range(
                    max(cancel_start.day, 1) if cancel_start.month == month else 1,
                    min(cancel_end.day, last_day) + 1 if cancel_end.month == month else last_day + 1
                ):
                    mess_schedule[day] = "No Mess"

    # Display the schedule
    print(f"\nMess schedule for {year}-{month:02d}:")
    for day in range(1, last_day + 1):
        mess_name = mess_schedule[day]
        rate = mess_rates.get(mess_name, 0) if mess_name != "No Mess" else 0
        print(f"  Day {day:02d}: {mess_name} - {rate} Rs")

# Calculate monthly bill of registered mess
def calculate_monthly_bill(user_id, mess_rates, registrations, cancellations):
    try:
        year = int(input("Enter the year (YYYY): "))
        month = int(input("Enter the month (MM): "))
    except ValueError as e:
        print(f"Invalid year or month input: {e}")
        return
    
    # Get the last day of the specified month
    try:
        _, last_day = monthrange(year, month)
    except (ValueError, IndexError) as e:
        print(f"Invalid month or year entered: {e}")
        return
    
    # Initializing the month's mess schedule with "No Mess"
    mess_schedule = {day: "No Mess" for day in range(1, last_day + 1)}
    
    # View registered mess data
    for reg in registrations:
        if reg["user_id"] == user_id:
            start_date = reg.get("start_date")
            end_date = reg.get("end_date")
            mess_name = reg.get("mess", "No Mess")
            
            if start_date and end_date:
                # Check if the registration overlaps with the specified month
                if start_date.year == year and start_date.month == month:
                    for day in range(max(start_date.day, 1), min(end_date.day, last_day) + 1):
                        mess_schedule[day] = mess_name
                elif start_date < datetime(year, month, last_day).date() and end_date > datetime(year, month, 1).date():
                    # Registration spans into the specified month
                    start_day = 1 if start_date.month < month else start_date.day
                    end_day = last_day if end_date.month > month else end_date.day
                    for day in range(start_day, end_day + 1):
                        mess_schedule[day] = mess_name
    
    # Apply cancellations of mess
    for cancel in cancellations:
        if cancel["user_id"] == user_id:
            cancel_start = cancel["start_date"]
            cancel_end = cancel["end_date"]
            
            if cancel_start.year == year and cancel_start.month == month:
                for day in range(max(cancel_start.day, 1), min(cancel_end.day, last_day) + 1):
                    mess_schedule[day] = "No Mess"
            elif cancel_start < datetime(year, month, last_day).date() and cancel_end > datetime(year, month, 1).date():
                # Cancellation spans into the specified month
                start_day = 1 if cancel_start.month < month else cancel_start.day
                end_day = last_day if cancel_end.month > month else cancel_end.day
                for day in range(start_day, end_day + 1):
                    mess_schedule[day] = "No Mess"

    # Calculate the monthly bill
    total_bill = 0
    bill_details = {}
    for day in range(1, last_day + 1):
        mess_name = mess_schedule[day]
        if mess_name != "No Mess":
            rate = mess_rates.get(mess_name, 0)
            total_bill += rate
            if mess_name in bill_details:
                bill_details[mess_name] += 1
            else:
                bill_details[mess_name] = 1

    # Display the monthly bill
    print(f"\nMonthly Bill for {year}-{month:02d}:")
    print("Mess Usage:")
    for mess, days in bill_details.items():
        rate = mess_rates.get(mess, 0)
        print(f"  {mess}: {days} days x {rate} Rs = {days * rate} Rs")
    print(f"Total Bill: {total_bill} Rs")

# File name for storing notifications
notifications_file = "notifications.csv"

# Initialize the notifications file
def initialize_notifications_file():
    try:
        with open(notifications_file, "x", newline="") as file:
            writer = csv.writer(file)
            # Write the header
            writer.writerow(["sender_type", "sender_name", "recipient_type", "recipient", "message", "timestamp"])
    except FileExistsError:
        pass

# Send a notification
def send_notification(sender_type, sender_name, users):
    recipient_type = input("Enter recipient type (mess staff, mess admin, All): ").lower()
    # Restrict recipient based on sender
    if sender_type == "student":
        if recipient_type not in {"mess staff", "mess admin"}:
            print("Students can only send messages to mess staff, mess admin")
            return
    elif sender_type in {"mess staff", "mess admin"}:
        if recipient_type not in {"mess staff", "mess admin", "student", "all"}:
            print("Invalid recipient type.")
            return
    else:
        print("You do not have permission to send messages.")
        return

    recipient = input("Enter recipient's User ID/role or 'All': ")
    if recipient != 'All':
        # Check if recipient exists and matches recipient_type
        if recipient not in users or users[recipient]["role"] != recipient_type:
            print("Recipient not found or role mismatch.")
            return
    message = input("Enter your message: ")
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Add the notifications to the CSV file
    with open(notifications_file, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([sender_type, sender_name, recipient_type, recipient, message, timestamp])
    
    print("Notification sent successfully!")

# Handle notifications based on the user's role
def handle_notifications(role, user_id, users):
    while True:
        print("\n--- Notifications Menu ---")
        print("1. Send Notification")
        print("2. Read Notifications")
        print("3. Back")
        choice = input("Enter your choice: ")

        if choice == "1":
            send_notification(role, user_id, users)
        elif choice == "2":
            read_notifications(role, user_id, users)
        elif choice == "3":
            break
        else:
            print("Invalid choice. Please try again.")

# Read notifications for a user
def read_notifications(user_type, user_id, users):
    print("\n--- Notifications ---")
    with open(notifications_file, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            sender_type = row["sender_type"]
            recipient_type = row["recipient_type"]
            recipient = row["recipient"]
            # Determine if the notification is visible to the user
            if recipient == 'All':
                if user_type in {"student", "mess staff", "mess admin"}:
                    print(f"[{row['timestamp']}] {row['sender_type']} ({row['sender_name']}): {row['message']}")
            elif recipient_type == user_type and recipient == user_id:
                print(f"[{row['timestamp']}] {row['sender_type']} ({row['sender_name']}): {row['message']}")
            elif recipient_type == user_type and recipient == 'All':
                print(f"[{row['timestamp']}] {row['sender_type']} ({row['sender_name']}): {row['message']}")
            elif user_type in {"mess staff", "mess admin"} and recipient_type in {"mess staff", "mess admin"} and recipient == 'All':
                # Mess staff and admin can see messages to mess staff/admin
                print(f"[{row['timestamp']}] {row['sender_type']} ({row['sender_name']}): {row['message']}")
    print("---------------------\n")

def view_registrations_for_mess(mess_name, selected_date):
    """View registrations for a specific mess on a given date."""
    from datetime import datetime
    
    # Ensure selected_date is a datetime.date object
    try:
        selected_date = datetime.strptime(selected_date, "%Y-%m-%d").date()
    except ValueError:
        print("Invalid format. Please use YYYY-MM-DD.")
        return

    print(f"\n--- Registrations for {mess_name} on {selected_date} ---")
    found = False
    seen_user_ids = set()  
    for reg in registrations:
        start_date = reg["start_date"]  
        end_date = reg["end_date"]      

        if reg["mess"] == mess_name and start_date <= selected_date <= end_date:
            user_id = reg["user_id"]
            
            # Check if this user has already been printed
            if user_id not in seen_user_ids:
                seen_user_ids.add(user_id)
                print(f"User ID: {user_id}")
                found = True

    if not found:
        print(f"No registrations found for {mess_name} on {selected_date}.")

def main():
    # Load data from CSV files
    users = load_users_from_csv()
    mess_data = load_menu_from_csv()
    mess_rates = load_mess_rates_from_csv()
    load_registrations_from_csv()
    load_cancellations_from_csv()
    initialize_notifications_file()  # Ensure notifications.csv is initialized

    # Assign default mess if not already registered
    for user_id in users:
        # Find the registration for this user 
        user_registration = next((reg for reg in registrations if reg["user_id"] == user_id), None)
        
        # Display the default mess for the user
        default_mess = DEFAULT_MESS.get(user_id, "North Mess")
        
        # If the user doesn't registered for the mess, assign the default mess
        if not user_registration:
            today = datetime.now().date()
            first_day = today.replace(day=1)
            last_day = (first_day + timedelta(days=31)).replace(day=1) - timedelta(days=1)
            
            default_registration = {
                "user_id": user_id,
                "mess": default_mess,
                "registration_type": "default",
                "start_date": first_day,
                "end_date": last_day,
            }
            registrations.append(default_registration)
            print(f"Assigned default mess '{default_mess}' to user '{user_id}' for {first_day.strftime('%B %Y')}.")
        
        # If the user already has a registration, check if it's correct and update 
        elif user_registration["mess"] != default_mess:
            user_registration["mess"] = default_mess
            print(f"Updated mess for user '{user_id}' to '{default_mess}'.")

    role, user_id = login(users)
    
    if role == "student":
        # Fetch default mess from registrations
        user_registrations = [reg for reg in registrations if reg["user_id"] == user_id]
        if user_registrations:
            # Assuming the first registration is the default mess
            default_mess = user_registrations[0]["mess"]
        else:
            default_mess = "North Mess"  
        print(f"Default mess for {user_id}: {default_mess}")
        
        while True:
            print("\n--- Student Menu ---")
            print("1. View Mess Menus")
            print("2. Register for Mess")
            print("3. Cancel Registration")
            print("4. View Mess for the Month")
            print("5. Calculate Monthly Bill")
            print("6. Notifications")
            print("7. Logout")
            choice = timed_input("Enter your choice: ")
            
            if choice == "1":
                view_mess_menu(mess_data)
            elif choice == "2":
                register_for_mess(user_id, mess_data)
            elif choice == "3":
                cancel_registration(user_id)
            elif choice == "4":
                view_mess_for_month(user_id, default_mess, mess_rates)
            elif choice == "5":
                calculate_monthly_bill(user_id, mess_rates, registrations, cancellations)
            elif choice == "6":
                handle_notifications("student", user_id, users)
            elif choice == "7":
                print("Logging out...")
                break
            else:
                print("Invalid choice. Please try again.")

    elif role == "mess staff" or role == "mess admin":
        print(f"Welcome, {role.capitalize()}!")
        while True:
            print(f"\n--- {role.capitalize()} Menu ---")
            print("1. View Mess Menus")
            print("2. Send Notifications")
            print("3. Read Notifications")
            print("4. View student registration for any mess")
            print("5. Logout")
            choice = timed_input("Enter your choice: ")

            if choice == "1":
                view_mess_menu(mess_data)
            elif choice == "2":
                send_notification(role, user_id, users)
            elif choice == "3":
                read_notifications(role, user_id, users)
            elif choice == "4":
                mess_name = input("Enter the mess name (e.g., 'Kadamba'): ")
                selected_date = input("Enter the date to view registrations (YYYY-MM-DD): ")
                view_registrations_for_mess(mess_name, selected_date)
            elif choice == "5":
                print("Logging out...")
                break
            else:
                print("Invalid choice. Please try again.")

    else:
        print("Access restricted to registered users only.")
    
    # Save registrations and cancellations to CSV files after updating all necessary data
    save_registrations_to_csv()
    save_cancellations_to_csv()

# Run the program
if __name__ == "__main__":main()
